In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

from torch.distributions.log_normal import LogNormal

# Internal packages
from data_loader import KidneyStoneDataset, ToTensor
from model import binary_ks_net, binary_neg_loglik, cont_rec_ks_net, cont_rec_neg_loglik, cont_size_ks_net, cont_size_neg_loglik
from train import train

## Gamma generative process with log-normal parametrization

In [ ]:
# Hyperparameters
BATCH_SIZE = 128
EPOCHS     = 150
LEARN_R    = 1e-2
N_HU       = 3

# Initialize the dataset
data = KidneyStoneDataset("./data/ks_cont_size_data_g.npy", transform=ToTensor())
train_loader = DataLoader(data, batch_size=BATCH_SIZE)

# Initialize the model
model = cont_size_ks_net(N_HU)

# Optimizers
#optimizer = optim.SGD(model.parameters(), lr=LEARN_R, weight_decay=0.1)
optimizer = optim.RMSprop(model.parameters(), lr=LEARN_R)

In [ ]:
cum_loss = train(model, optimizer, cont_size_neg_loglik, train_loader, EPOCHS)

In [ ]:
# First, we get the parameters of the size variable:
n_samples = 20000
arbitrary_query = torch.tensor([1., 1., 1.]) # It is only important that the first element is 1
mu_L, log_sigma_L, _, _, _ = model(arbitrary_query.unsqueeze(0))
sigma_L = torch.exp(log_sigma_L)
L_dist = LogNormal(mu_L, sigma_L)
L_samples = L_dist.sample((n_samples,)).view(n_samples,1)
L_prob = torch.exp(L_dist.log_prob(L_samples))

In [ ]:
t_1 = torch.cat((L_samples, torch.ones(n_samples, 2)), 1)
t_0 = torch.cat((L_samples, torch.zeros(n_samples, 1) ,torch.ones(n_samples, 1)), 1)

In [ ]:
# Probabilities with Treatment A
_, _, _, mu_t1, log_sigma_t1 = model(t_1) # Probabilities with KS = L
_, _, _, mu_t0, log_sigma_t0 = model(t_0) # Probabilities with KS = L

In [ ]:
# By monte carlo integration, we are getting mu from the value of the variables rom the distribution
torch.mean(mu_t1) - torch.mean(mu_t0) # lognormal 1 sample 4.1766, 5 samples 4.1110, 20 samples 4.0318, 50 samples 4.1502, 200 samples 4.1494, 2k samples 4.1117

## Log-normal generative process with log-normal parametrization

In [2]:
# Hyperparameters
BATCH_SIZE = 128
EPOCHS     = 150
LEARN_R    = 1e-2
N_HU       = 3

# Initialize the dataset
data = KidneyStoneDataset("./data/ks_cont_size_data_ln.npy", transform=ToTensor())
train_loader = DataLoader(data, batch_size=BATCH_SIZE)

# Initialize the model
model = cont_size_ks_net(N_HU)

# Optimizers
#optimizer = optim.SGD(model.parameters(), lr=LEARN_R, weight_decay=0.1)
optimizer = optim.RMSprop(model.parameters(), lr=LEARN_R)

In [3]:
cum_loss = train(model, optimizer, cont_size_neg_loglik, train_loader, EPOCHS)

In [32]:
# First, we get the parameters of the size variable:
n_samples = 20000
arbitrary_query = torch.tensor([1., 1., 1.]) # It is only important that the first element is 1
mu_L, log_sigma_L, _, _, _ = model(arbitrary_query.unsqueeze(0))
sigma_L = torch.exp(log_sigma_L)
L_dist = LogNormal(mu_L, sigma_L)
L_samples = L_dist.sample((n_samples,)).view(n_samples,1)
L_prob = torch.exp(L_dist.log_prob(L_samples))

In [33]:
t_1 = torch.cat((L_samples, torch.ones(n_samples, 2)), 1)
t_0 = torch.cat((L_samples, torch.zeros(n_samples, 1) ,torch.ones(n_samples, 1)), 1)

In [34]:
# Probabilities with Treatment A
_, _, _, mu_t1, log_sigma_t1 = model(t_1) # Probabilities with KS = L
_, _, _, mu_t0, log_sigma_t0 = model(t_0) # Probabilities with KS = L

In [35]:
# By monte carlo integration, we are getting mu from the value of the variables rom the distribution
torch.mean(mu_t1) - torch.mean(mu_t0) # lognormal 1 sample 4.1766, 5 samples 4.1110, 20 samples 4.0318, 50 samples 4.1502, 200 samples 4.1494, 2k samples 4.1117

tensor(3.6989, grad_fn=<SubBackward0>)